In [2]:
from PIL import Image
import itertools
import numpy as np
import pandas as pd
import warnings
import nltk
import math
import time
import re
import os
import pickle
#Deep-Learning Library
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import glob

Using TensorFlow backend.


In [3]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = "/content/drive/My Drive/Major Project B.Tech Sem 8/Breast Cancer Dataset/"
savingpath="/content/drive/My Drive/Major Project B.Tech Sem 8/"

In [0]:
train_data_dir=path+"Train"
test_data_dir=path+"Test"

#Training paths
data_cnn_VGG16_train=savingpath+"VGG16/train/data_cnn_VGG16_train.npy"
data_filenames_VGG16_train=savingpath+"VGG16/train/data_filenames_VGG16_train.npy"
data_cancerclass_VGG16_train=savingpath+"VGG16/train/data_cancerclass_VGG16_train.npy"
data_cancertype_VGG16_train=savingpath+"VGG16/train/data_cancertype_VGG16_train.npy"
data_mag_VGG16_train=savingpath+"VGG16/train/data_mag_VGG16_train.npy"

#Testing paths
data_cnn_VGG16_test=savingpath+"VGG16/test/data_cnn_VGG16_test.npy"
data_filenames_VGG16_test=savingpath+"VGG16/test/data_filenames_VGG16_test.npy"
data_cancerclass_VGG16_test=savingpath+"VGG16/test/data_cancerclass_VGG16_test.npy"
data_cancertype_VGG16_test=savingpath+"VGG16/test/data_cancertype_VGG16_test.npy"
data_mag_VGG16_test=savingpath+"VGG16/test/data_mag_VGG16_test.npy"

In [6]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np

#model = VGG16(weights='imagenet', include_top=True)

base_model = VGG16(weights='imagenet',include_top=True)
model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
image_size = (224, 224)


Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 6s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  # Remove the CWD from sys.path while we load stuff.


In [8]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 224, 224
#top_model_weights_path = 'bottleneck_fc_model.h5'
#epochs = 50
#batch_size = 1
print("Started")

def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return "NO" 
    else: 
        return "YES" 

def save_bottlebeck_features_train(train_data=train_data_dir,nb_train_samples=5000,batch_size=1):
    
    #Function to compute VGG-16 CNN for image feature extraction.
        
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    features=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the VGG16 network
    #model = applications.VGG16(include_top=False, weights='imagenet')
    #print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    a=1
    for i in generator.filenames:
        #print(i)
        #print(type)
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        k=0
        for types in cancersubtypes:          
          flagforcancersubtypes=check(i,types)         
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(flagforcancersubtypes=="YES"):
            cancertype.append(codeforcancersubtypes[k])
          k=k+1
         
        # Array for Magnification
        magnificationsformat=["-40-","-100-","-200-","-400-"]
        codeformagnifications=[40,100,200,400]
        j=0
        for types in magnificationsformat:
          
          flagformagnifications=check(i,types)         
          if(flagformagnifications=="YES"):
            mag.append(codeformagnifications[j])
          j=j+1
 
        image_path=train_data_dir+"/"+i
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
    
    print(features)
        
    print(cancerclass)    
    print(cancertype)
    print(mag)
    print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    
    features=np.array(features)
    print(features.shape)
     
    
    print("reshape the trained vector")
    #features = features.reshape((5000,25088))
    print("Trainning done")
    np.save(open(data_cnn_VGG16_train, 'wb'), features)
    np.save(open(data_filenames_VGG16_train, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_VGG16_train, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_VGG16_train, 'wb'), np.array(cancertype))
    np.save(open(data_mag_VGG16_train, 'wb'), np.array(mag))
    print("npy file saved")

save_bottlebeck_features_train()

print(time.clock() - start_time, "seconds")

Started
Initialized Required data and datagen done
Found 5000 images belonging to 1 classes.
Genrator work finish
[array([0.       , 0.       , 1.962454 , ..., 0.       , 0.4253466,
       0.       ], dtype=float32), array([0.      , 0.      , 8.675269, ..., 0.      , 0.      , 0.      ],
      dtype=float32), array([0.      , 0.      , 2.862421, ..., 0.      , 0.      , 0.      ],
      dtype=float32), array([0.       , 0.       , 4.6714525, ..., 0.       , 0.       ,
       2.4155934], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), array([0.       , 0.       , 0.       , ..., 0.       , 6.0846763,
       0.       ], dtype=float32), array([0.      , 0.      , 0.      , ..., 0.      , 0.      , 5.474881],
      dtype=float32), array([0.       , 0.       , 7.4887013, ..., 0.       , 0.       ,
       0.7236998], dtype=float32), array([0.       , 0.       , 7.11

In [9]:
start_time = time.clock()
# dimensions of our images.
img_width, img_height = 224, 224
top_model_weights_path = 'bottleneck_fc_model.h5'
epochs = 50
batch_size = 1
print("Started")

def check(string, sub_str): 
    if (string.find(sub_str) == -1): 
        return "NO" 
    else: 
        return "YES" 

def save_bottlebeck_features_test(train_data=test_data_dir,nb_train_samples=2900,batch_size=1):
    
    #Function to compute VGG-16 CNN for image feature extraction.
    
    features=[]
    filenames = []
    cancerclass=[]
    cancertype=[]
    mag=[]
    datagen = ImageDataGenerator(rescale=1. / 255)
    print("Initialized Required data and datagen done")                                                
    # build the VGG16 network
    #model = applications.VGG16(include_top=False, weights='imagenet')
    #print("Model settled")
    generator = datagen.flow_from_directory(
        train_data,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    print("Genrator work finish")
    start_time = time.clock()
    for i in generator.filenames:
        
        # Array for File NAmes
        filenames.append(i)
        
        # Array For Cancer Class
        X=re.search("_B_", i)
        if (X):
          cancer=1
          cancerclass.append(cancer)
        else:
          cancer=2
          cancerclass.append(cancer)
        
        # Array For Cancer Type
        cancersubtypes=['_A-','_F-','_PT-','_TA-','_DC-','_LC-','_MC-','_PC-']
        codeforcancersubtypes=[11,12,13,14,21,22,23,24]
        k=0
        for types in cancersubtypes:          
          flagforcancersubtypes=check(i,types)         
          #print(type(pattern.match(i)))
          #print(obtainedpattern)
          if(flagforcancersubtypes=="YES"):
            cancertype.append(codeforcancersubtypes[k])
          k=k+1
         
        # Array for Magnification
        magnificationsformat=["-40-","-100-","-200-","-400-"]
        codeformagnifications=[40,100,200,400]
        j=0
        for types in magnificationsformat:
          
          flagformagnifications=check(i,types)         
          if(flagformagnifications=="YES"):
            mag.append(codeformagnifications[j])
          j=j+1
        image_path=test_data_dir+"/"+i
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
    
        
    print(cancerclass)    
    print(cancertype)
    print(mag)
    print("For loop ended")
    print(time.clock() - start_time, "seconds, time for loop")
    print(generator)
    
    print("Trainning done")
    np.save(open(data_cnn_VGG16_test, 'wb'), features)
    np.save(open(data_filenames_VGG16_test, 'wb'), np.array(filenames))
    np.save(open(data_cancerclass_VGG16_test, 'wb'), np.array(cancerclass))
    np.save(open(data_cancertype_VGG16_test, 'wb'), np.array(cancertype))
    np.save(open(data_mag_VGG16_test, 'wb'), np.array(mag))
    print("npy file saved")
    

save_bottlebeck_features_test()

print(time.clock() - start_time, "seconds , Testing data also saved")

Started
Initialized Required data and datagen done
Found 2900 images belonging to 1 classes.
Genrator work finish
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 